In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from tensorflow.keras import layers, models
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
import tensorflow as tf
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


/bin/python3.11


In [4]:
class Classifier:
    def __init__(self, encoder, input_shape, num_classes):
        self.encoder = encoder
        self.model = self._build_classifier(input_shape, num_classes)
    
    def _build_classifier(self, input_shape, num_classes):
        inputs = layers.Input(shape=input_shape)

        # Freeze encoder weights
        self.encoder.trainable = False

        # Ensure the input has the correct shape for the encoder (e.g., (500, 1) for FCMAE)
        x = self.encoder(inputs, training=False)  # Use frozen encoder

        # Flatten the output of the encoder (if necessary)
        x = layers.Flatten()(x)

        # Add dense layers for classification
        x = layers.Dense(32, activation="relu")(x)
        x = layers.Dense(16, activation="relu")(x)
        outputs = layers.Dense(num_classes, activation="softmax")(x)

        model = models.Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy"])
        return model

    def train(self, x_train, y_train, batch_size=64, epochs=5, validation_split=0.1):
        history = self.model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split)
        return history
    
    def evaluate(self, x_test, y_test):
        return self.model.evaluate(x_test, y_test)
    
    def predict(self, x):
        return self.model.predict(x)

# Load the trained encoder from MaskedAutoencoder
encoder_path = "../models/encoder_model.keras"
if os.path.exists(encoder_path):
    # Load saved encoder
    print("Loading saved encoder...")
    encoder = tf.keras.models.load_model(encoder_path)
else:
    # Train autoencoder and extract encoder
    autoencoder = MaskedAutoencoder(input_dim=500)
    input_layer = layers.Input(shape=(500,))
    encoded_output = autoencoder.encoder(input_layer)
    encoder = models.Model(input_layer, encoded_output)

    # Save encoder
    print("Saving trained encoder...")
    encoder.save(encoder_path)

# Load the target dataset
save_dir = os.path.join(univariate_data_path, "target_univariate.npy")
target_data = np.load(save_dir, allow_pickle=True)

# Create windows for training
def create_windows(data, window_size=500, step=250):
    all_windows, labels = [], []
    for sample in data:
        signal, label = sample["signal"], sample["preterm"]
        
        if label is None:  # Skip instances with None labels (just in case)
            continue
        
        for start in range(0, len(signal) - window_size + 1, step):
            window = signal[start : start + window_size]
            all_windows.append(window)
            labels.append(label)

    return np.array(all_windows, dtype=np.float32), np.array(labels, dtype=np.int32)

# Adjust input shape for FCMAE (e.g., (500, 1) for each window)
x_train, y_train = create_windows(target_data)  # Make sure this function is called!

# Reshape the data if needed for FCMAE (e.g., (500,) -> (500, 1))
x_train = x_train.reshape((-1, 500, 1))  # Reshaping to match FCMAE input

# Initialize and train the classifier
num_classes = len(set(y_train))  # Get number of unique classes
classifier = Classifier(encoder, input_shape=(500, 1), num_classes=num_classes)

# Train the classifier
classifier.train(x_train, y_train)


Loading saved encoder...
Epoch 1/5
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.7371 - loss: 0.5459 - val_accuracy: 0.7137 - val_loss: 0.6403
Epoch 2/5
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.7418 - loss: 0.5302 - val_accuracy: 0.7025 - val_loss: 0.6410
Epoch 3/5
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.7422 - loss: 0.5264 - val_accuracy: 0.7116 - val_loss: 0.6353
Epoch 4/5
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.7438 - loss: 0.5228 - val_accuracy: 0.7128 - val_loss: 0.6270
Epoch 5/5
8562/8562 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.7435 - loss: 0.5204 - val_accuracy: 0.7129 - val_loss: 0.6439
